In [32]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 3000
nb_validation_samples = 1000
epochs = 50
batch_size = 20

In [33]:
def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy', 'rb'))
    train_labels = np.array(
        [0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    validation_data = np.load(open('bottleneck_features_validation.npy', 'rb'))
    validation_labels = np.array(
        [0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)


In [34]:
save_bottlebeck_features()

Found 3000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [35]:
train_top_model()

Train on 3000 samples, validate on 1000 samples
Epoch 1/50
3000/3000 [==============================] - 1s 366us/step - loss: 0.6304 - acc: 0.7803 - val_loss: 0.2799 - val_acc: 0.8790
Epoch 2/50
3000/3000 [==============================] - 1s 260us/step - loss: 0.3287 - acc: 0.8613 - val_loss: 0.2593 - val_acc: 0.8880
Epoch 3/50
3000/3000 [==============================] - 1s 257us/step - loss: 0.2849 - acc: 0.8880 - val_loss: 0.2645 - val_acc: 0.8910
Epoch 4/50
3000/3000 [==============================] - 1s 255us/step - loss: 0.2610 - acc: 0.8997 - val_loss: 0.2631 - val_acc: 0.8880
Epoch 5/50
3000/3000 [==============================] - 1s 254us/step - loss: 0.2157 - acc: 0.9180 - val_loss: 0.3008 - val_acc: 0.8810
Epoch 6/50
3000/3000 [==============================] - 1s 260us/step - loss: 0.1931 - acc: 0.9263 - val_loss: 0.3479 - val_acc: 0.8810
Epoch 7/50
3000/3000 [==============================] - 1s 260us/step - loss: 0.1782 - acc: 0.9310 - val_loss: 0.3308 - val_acc: 0.8830
